# Step 10: V07 — ExoFOP TOI lookup (context)

Goal: confirm the candidate exists in ExoFOP and capture community context (disposition, notes, and basic parameters).

This is a catalog-context check (network required).


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

ds = sh.load_dataset()
lc = sh.stitch_pdcsap(ds)
depth_ppm, _ = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

session = sh.make_session(
    stitched=lc,
    candidate=candidate,
    network=True,
    ra_deg=sh.RA_DEG,
    dec_deg=sh.DEC_DEG,
)

r = session.run('V07')

print(
    json.dumps(
        {
            'status': r.status,
            'flags': r.flags,
            'metrics': r.metrics,
        },
        indent=2,
        sort_keys=True,
    )
)


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "flags": [
    "EXOFOP_MATCH"
  ],
  "metrics": {
    "cache_age_seconds": 20.829258918762207,
    "cache_stale": false,
    "cache_ttl_seconds": 86400,
    "fetch_error": null,
    "found": true,
    "source": "exofop_toi_table",
    "status": "ok",
    "tic_id": 188646744,
    "toi": "5807.01",
    "used_stale_cache": false
  },
  "status": "ok"
}
```

</details>


In [ ]:
# Plot V07: ExoFOP TOI card
out = {
    'status': r.status,
    'flags': r.flags,
}

try:
    import matplotlib.pyplot as plt
    from bittr_tess_vetter.api import plot_exofop_card
    PLOTTING_AVAILABLE = True
except Exception as e:
    PLOTTING_AVAILABLE = False
    out['plotting_error'] = str(e)

if PLOTTING_AVAILABLE and (r.status == 'ok') and getattr(r, 'raw', None) and (r.raw or {}).get('plot_data'):
    run_out_dir, docs_out_dir = sh.artifact_dirs(step_id='10_v07_exofop')
    run_path = run_out_dir / 'V07_exofop_card.png'
    docs_path = (docs_out_dir / 'V07_exofop_card.png') if docs_out_dir is not None else None

    fig, ax = plt.subplots(figsize=(5, 4))
    plot_exofop_card(r, ax=ax)
    ax.set_title('V07: ExoFOP TOI lookup')
    fig.tight_layout()

    fig.savefig(run_path, dpi=150, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=150, bbox_inches='tight')
    plt.show()

    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


**Pre-rendered plot (no execution required):** `../artifacts/tutorial_toi-5807-incremental/10_v07_exofop/V07_exofop_card.png`

![V07: ExoFOP TOI lookup](../artifacts/tutorial_toi-5807-incremental/10_v07_exofop/V07_exofop_card.png)


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/10_v07_exofop/V07_exofop_card.png",
  "flags": [
    "EXOFOP_MATCH"
  ],
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/10_v07_exofop/V07_exofop_card.png",
  "status": "ok"
}
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Flags:** EXOFOP_MATCH.
- **Result:** ExoFOP lookup found the TOI entry (flag `EXOFOP_MATCH`).
- **Why it’s useful:** provides community parameters and follow-up hooks (e.g. ExoFOP depth ∼225 ppm and Rp ∼2.12 R⊕ for context).
- **Interpretation:** use ExoFOP for context, but keep the tutorial’s offline ephemeris/depth as the analysis baseline.
- **Next step:** pixel localization (V08–V10) using per-sector TPFs.

</details>
